In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.ne

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-23 02:13:14--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.18MB/s    in 0.8s    

2022-12-23 02:13:16 (1.18 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [4]:
# Start spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ETL_final_project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
import pandas as pd
from pyspark import SparkFiles

In [6]:
# Drive helper
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# path ="/content/drive/MyDrive/data/Daily_County-Level_Ozone_Concentrations__2001-2016.csv"
# spark.sparkContext.addFile(path)
# df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
# df.show()

In [8]:
path ="/content/drive/MyDrive/data/Daily_County-Level_Ozone_Concentrations__2001-2016.csv"
ozone_df= pd.read_csv(path)
ozone_df.tail()


,Year,Month,Day,statefips,countyfips,O3_max_pred,O3_med_pred,O3_mean_pred,O3_pop_pred
18168991,2016,DEC,31,56,37,41.8023,40.98200,40.954517,41.030787
18168992,2016,DEC,31,56,39,39.8256,39.24925,39.252300,39.160351
18168993,2016,DEC,31,56,41,42.7244,39.59090,40.449267,40.374935
18168994,2016,DEC,31,56,43,35.3622,33.30220,33.924567,34.047683
18168995,2016,DEC,31,56,45,39.6921,38.58465,38.584650,38.673762


In [9]:
path2="/content/drive/MyDrive/data/cancer_incidence.csv"
df_cancer=pd.read_csv(path2)
pd.set_option('display.max_columns', None)
df_cancer.head()

,index,County,FIPS,"Age-Adjusted Incidence Rate(Ê) - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,Average Annual Count,Recent Trend,Recent 5-Year Trend (ˆ) in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1
0,0,"US (SEER+NPCR)(1,10)",0,62.4,62.3,62.6,214614,falling,-2.5,-3,-2
1,1,"Autauga County, Alabama(6,10)",1001,74.9,65.1,85.7,43,stable,0.5,-14.9,18.6
2,2,"Baldwin County, Alabama(6,10)",1003,66.9,62.4,71.7,170,stable,3,-10.2,18.3
3,3,"Barbour County, Alabama(6,10)",1005,74.6,61.8,89.4,25,stable,-6.4,-18.3,7.3
4,4,"Bibb County, Alabama(6,10)",1007,86.4,71,104.2,23,stable,-4.5,-31.4,32.9


In [10]:
# Remove years after 2014
ozone_df= ozone_df[ozone_df['Year']<=2014]
ozone_df.tail()

,Year,Month,Day,statefips,countyfips,O3_max_pred,O3_med_pred,O3_mean_pred,O3_pop_pred
15896312,2014,DEC,31,56,37,32.9847,31.53745,31.483275,31.626598
15896313,2014,DEC,31,56,39,36.7321,35.82770,35.939625,35.840333
15896314,2014,DEC,31,56,41,31.3784,31.29280,31.166967,31.181609
15896315,2014,DEC,31,56,43,30.0499,28.02510,28.673833,28.789780
15896316,2014,DEC,31,56,45,38.0500,37.32785,37.327850,37.385959


In [11]:
ozone_df.dtypes

Year              int64
Month            object
Day               int64
statefips         int64
countyfips        int64
O3_max_pred     float64
O3_med_pred     float64
O3_mean_pred    float64
O3_pop_pred     float64
dtype: object

In [17]:
ozone_df=ozone_df.astype({'statefips':'string', 'countyfips':'string'})


In [18]:
ozone_df.dtypes

Year              int64
Month            object
Day               int64
statefips        string
countyfips       string
O3_max_pred     float64
O3_med_pred     float64
O3_mean_pred    float64
O3_pop_pred     float64
FIPS              int64
dtype: object

In [19]:
ozone_df ['FIPS']= ozone_df['statefips'].astype(str) + ozone_df['countyfips']
ozone_df.head()

,Year,Month,Day,statefips,countyfips,O3_max_pred,O3_med_pred,O3_mean_pred,O3_pop_pred,FIPS
0,2001,JAN,1,1,1,31.939058,31.691988,31.680859,31.671226,11
1,2001,JAN,1,1,3,33.646855,33.170271,32.994775,32.935070,13
2,2001,JAN,1,1,5,34.288917,34.068507,34.077954,34.086631,15
3,2001,JAN,1,1,7,30.349767,30.036093,29.931756,29.991733,17
4,2001,JAN,1,1,9,26.472655,25.776595,25.857571,25.872472,19
